# 작사가 인공지능 만들기

# Step1. 데이터 불러오기

In [ ]:
# glob모듈을 이용하여 모든 txt파일을 읽어와서 raw_corpus리스트에 문장 단위로 저장하겠습니다.
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = '/content/drive/MyDrive/익스플러레이션 데이터 - 아이펠 /lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me', 'Cause you make me feel so brand new', "And I want to spend my life with you Let me say that since, baby, since we've been together", 'Loving you forever', 'Is what I need', 'Let me, be the one you come running to', "I'll never be untrue Oh baby", "Let's, let's stay together (gether)", "Lovin' you whether, whether", 'Times are good or bad, happy or sad', 'Oh, oh, oh, oh, yeah', 'Whether times are good or bad, happy or sad Why, why some people break up', 'Then turn around and make up', "I just can't see", "You'd never do that to me (would you, baby)", 'Staying around you is all I see', "(Here's what I want us do) Let's, we oughta stay together (gether)", 'Loving you whether, whether']


# Step2. 데이터 전처리 및 정제
문장을 토큰화(Tokenize)시키기전에 몇가지 문제가 되는 상황이 있습니다.
1. 문장부호
2. 대소문자
3. 특수문자

해당 문제들을 제거해주기위해 전처리함수를 정의합니다.

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

공백이나 화자등의 우리가 원하지않는 문장들을 정제해봅시다. 그리고 전처리도 같이 진행해줍니다.

In [ ]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> let s stay together i , i m i m so in love with you <end>',
 '<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 '<start> and i want to spend my life with you let me say that since , baby , since we ve been together <end>',
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 '<start> i ll never be untrue oh baby <end>',
 '<start> let s , let s stay together gether <end>']

# Step3. 토근화
텐서플로우의 토큰화패키지는 정제된 데이터를 토큰화해주고, 단어사전을 만들어주며, 데이터를 숫자로 변환까지 한 방에 해줍니다. 이 과정을 **벡터화**라하며, 숫자로 변환된 데이터를 **텐서**라고 부릅니다.

In [ ]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=13000,     # 13,000단어를 기억할 수 있는 tokenizer를 만들어줍니다.
        filters=' ',         # 이미 문장을 정제했으니 filters가 필요없습니다.
        oov_token="<unk>"    # 13,000단어에 포함되지 못한 단어는 '<unk>'로 바꿔줍니다.
    )
    
    tokenizer.fit_on_texts(corpus)    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    
    tensor = tokenizer.texts_to_sequences(corpus)    # 준비한tokenizer를 이용해 corpus를 Tensor로 변환합니다
  
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  63  16 ...   0   0   0]
 [  2 598   7 ...   0   0   0]
 [  2  26  24 ...   0   0   0]
 ...
 [  2  35 134 ...   0   0   0]
 [  2   5  61 ...   0   0   0]
 [  2 171   3 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fc78032c2d0>


생성된 텐서 데이터를 3번째행, 10번째 열까지만 출력해보겠습니다.

In [ ]:
print(tensor.shape)
print('')
print(tensor[:3, :10])

(175749, 347)

[[  2  63  16 222 283   5   4   5  22   5]
 [  2 598   7  62  10  47   3   0   0   0]
 [  2  26  24  84  31  12   3   0   0   0]]


텐서 데이터는 모두 정수로 이루어져있고, 이 숫자는 tokenizer에 구축된 단어 사전의 인덱스입니다. 단어 사전이 어떻게 구축되어있는지 아래와 같이 확인해봅시다.

In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


여기서 start는 인덱스2에해당하고 end는 인덱스3에 해당하는것을 확인할 수 있습니다.

너무 긴문장은 다른 데이터들이 과도한 padding을 갖게 하므로제거해줍니다.
토근의 개수가 15개를 넘어가는 문장을 데이터에서 제거하겠습니다.

In [ ]:
# 15개가 넘어가는 문장인 행제거
i = 0
try:
  while tensor.shape[0] !=  i:
    if tensor[i,15] != 0:
      tensor = np.delete(tensor, i, axis = 0)
    else:
      i += 1 
except:
  print("완료되었습니다.")

tensor.shape

(156013, 347)

In [ ]:
# 15번째열 넘어서부터는 전부 0이기때문에, 해당 열을 전부 제거 
while tensor.shape[1] != 15:
    tensor = np.delete(tensor, 15 , axis = 1)

tensor.shape

(156013, 15)

## 소스문장과 타겟문장 생성

In [ ]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 598   7  62  10  47   3   0   0   0   0   0   0   0]
[598   7  62  10  47   3   0   0   0   0   0   0   0   0]


소스문장이 처음과 끝이 2와3으로 끝나고, 타겟문장은 3으로 끝나는것을 확인할 수 있습니다.
# Step4. 데이터셋 객체 생성

텐서플로우를 활용할때 **데이터셋객체**는 데이터 입력 파이프라인을 통한 속도개선 및 각종 편의기능을 제공합니다.

In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1  # tokenizer가 구축한 단어사전 내 13,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 13,001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) # 준비한 데이터 소스로부터 데이터셋객체를 만듭니다
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

Embedding 계층 : 텐서의 인덱스값을 해당 인덱스번째의 워드 벡터로 바꿔줍니다. 이 워드벡터는 의미벡터공간에서 단어의 추상적표현으로 사용됩니다.
# Step5. 네트워크 구현


In [ ]:
# 계층 구현
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # 워드벡터의 차원수
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 13001), dtype=float32, numpy=
array([[[ 2.31591639e-05, -3.00065120e-04,  1.75035035e-04, ...,
          2.78022362e-05,  1.29447479e-04,  6.40035068e-05],
        [ 1.11434056e-04, -5.91128133e-04,  1.58930023e-04, ...,
          2.91204537e-06,  1.63713470e-04,  1.53851186e-04],
        [ 8.78944484e-05, -7.86744116e-04,  2.33918516e-04, ...,
          2.00838622e-04,  3.38028127e-04,  5.46143201e-05],
        ...,
        [ 4.77558438e-04, -2.10592421e-04, -2.07982594e-04, ...,
          4.98414331e-04,  9.18051344e-04, -6.76701253e-04],
        [ 4.98222944e-04, -1.02681428e-04, -3.00775515e-04, ...,
          7.33476074e-04,  1.50192517e-03, -9.43650899e-04],
        [ 5.28636621e-04, -1.35444971e-05, -3.85415682e-04, ...,
          9.89436405e-04,  2.04909313e-03, -1.22547452e-03]],

       [[ 2.31591639e-05, -3.00065120e-04,  1.75035035e-04, ...,
          2.78022362e-05,  1.29447479e-04,  6.40035068e-05],
        [ 3.74716692e-05, -4.59301751e-04,  2

In [ ]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  3328256   
                                                                 
 lstm_6 (LSTM)               multiple                  5246976   
                                                                 
 lstm_7 (LSTM)               multiple                  8392704   
                                                                 
 dense_3 (Dense)             multiple                  13326025  
                                                                 
Total params: 30,293,961
Trainable params: 30,293,961
Non-trainable params: 0
_________________________________________________________________


매개변수가 3천만개가 쓰였습니다.

# Step6. 모델 훈련시키기

In [ ]:
# 모델 학습시키기
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)


model.fit(dataset, epochs=15)

Epoch 1/15
609/609 [==============================] - 59s 94ms/step - loss: 3.4361
Epoch 2/15
609/609 [==============================] - 58s 95ms/step - loss: 2.9793
Epoch 3/15
609/609 [==============================] - 58s 94ms/step - loss: 2.8034
Epoch 4/15
609/609 [==============================] - 58s 94ms/step - loss: 2.6717
Epoch 5/15
609/609 [==============================] - 58s 95ms/step - loss: 2.5611
Epoch 6/15
609/609 [==============================] - 58s 94ms/step - loss: 2.4623
Epoch 7/15
609/609 [==============================] - 58s 94ms/step - loss: 2.3722
Epoch 8/15
609/609 [==============================] - 58s 94ms/step - loss: 2.2866
Epoch 9/15
609/609 [==============================] - 58s 94ms/step - loss: 2.2067
Epoch 10/15
609/609 [==============================] - 58s 94ms/step - loss: 2.1315
Epoch 11/15
609/609 [==============================] - 58s 94ms/step - loss: 2.0608
Epoch 12/15
609/609 [==============================] - 58s 95ms/step - loss: 1.9934
E

에포크를 진행할수록 loss가 낮아지는것을 확인할수 있습니다.

# Step7. 모델 평가해보기

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
  
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        
        predict = model(test_tensor)  # 1. 입력받은 문장의 텐서를 입력합니다
        
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)  # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다 
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> what are", max_len=20)

'<start> what are you waiting for wow ? <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i can", max_len=20)

'<start> i can t help it if i wanted to <end> '

사람이 직접봤을때 완벽하진않지만, 충분히 이해할 수 있는 문장이 만들어졌습니다.

# 회고
NLP는 처음다뤄봤기때문에 낯선개념들이 많이 등장해서 코드하나하나 상세하게 이해하지는 못했다..텐서부터 토근화, 각종 계층들 전부 새롭게 배운 개념들이다. 그리고 전처리과정에서 정규식을 사용해서 특정패턴의 문자열들을 변환시켜줬는데, 종종 정규식을 마주쳐왔었지만 상세코드는 이해하지않고 그냥넘어갔었다. 하지만 NLP를 하려면 정규식을 많이 공부해야한다는것을 느꼈다. 토큰화든 뭐든간에 반드시 전처리와 정제과정을 거쳐야하기때문이다. 그리고 너무 긴문장들은 가사에서 사용하기에 어울리지않기때문에 토큰이 15개 넘어가는 문장은 삭제를해주라는 조건이있었다. 간단한 조건이었지만, 부족한 실력탓에 꽤 시간을 많이썼다.. 행을 삭제하는 과정에서 인덱스에러가 계속 뜨길래 이해가안됬었다. 그래서 작은 단위로 계속 출력해서보니 행을 삭제하면 전체 shape의 행숫자가 바뀌는것이었다. 이거를 고려하지않아서 for반복문을 통해서 행을 지우다보니까 처음전체행보다 더 높은숫자의 행을 지우려해서 인덱스에러가뜬것이었다..알고리즘 공부도 꾸준히해야함을 느꼈다.낯선개념들이 많이 나와서 새롭게 공부해야할 양이 많았지만, 하나하나의 개념이 전부 이해못할정도로 어려운건아니었고, 마지막에 모델을 학습시키고 문장을 새로 뽑아줄때 나름의 재미와 신기함을 느꼈다. 